In [67]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

## Step 1: Load data

In [68]:
df_train = pd.read_csv('../experiments/train.csv')
df_test = pd.read_csv('../experiments/test.csv')

# for convenience
datasets = [df_train, df_test]

## Step 2: Basic cleaning and preprocessing

In [69]:
# Fill in missing values
mAge = pd.concat((df_train['Age'], df_test['Age']), axis=0).mean()
medFare = pd.concat((df_train['Fare'], df_test['Fare']), axis=0).median()
for df in datasets:
    df['Age'] = df['Age'].fillna(mAge)
    df['Fare'] = df['Fare'].fillna(medFare)
    df['Embarked'] = df['Embarked'].fillna('S')

## Step 3: Feature engineering

In [70]:
# find fare bins based on frequency rather than value
garbage, fare_bins = pd.qcut(df_train['Fare'].append(df_test['Fare']), 4, retbins=True)
print(fare_bins)
# loops through both train and test set for convenience
for df in datasets:
    # bin age
    df['Age_binned'] = pd.cut(df['Age'], [0,16,32,48,64,200], labels = [0,1,2,3,4], retbins=False)
    
    # bin fare
    df['Fare_binned'] = pd.cut(df['Fare'], fare_bins, labels = [0,1,2,3], include_lowest=True, retbins=False)
    
    # family features
    df['Family_size'] = df_train['SibSp'] + df_train['Parch']
    df['Is_Alone'] = (df['Family_size'] == 0).astype(int)

[  0.       7.8958  14.4542  31.275  512.3292]


In [71]:
# drop unneeded rows
for df in datasets:
    df.drop(['Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Name'], axis=1, inplace=True)
    
# need to keep passengerID for submission

## Step 4: Format data for modelling

In [72]:
# one-hot encode categoricals using pd.get_dummies
df_train = pd.get_dummies(df_train, columns=['Embarked', 'Sex'])
df_test = pd.get_dummies(df_test, columns=['Embarked', 'Sex'])

In [73]:
# Split data into X and y, and select features to use
X = df_train.drop(['Survived', 'PassengerId'], axis=1)
y = df_train['Survived']
X_test = df_test.drop(['PassengerId'], axis=1)

In [74]:
# Split data X and y into train and val sets
X_train, X_val, y_train, y_val = train_test_split(
      X, y, test_size=0.2, random_state=42)

In [75]:
df_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age_binned     0
Fare_binned    0
Family_size    0
Is_Alone       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
Sex_female     0
Sex_male       0
dtype: int64

## Step 5: Train (or fit) the model

In [76]:
# Fit logistic regression
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [77]:
# fit rf
rf = RandomForestClassifier(random_state=12)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [78]:
# fit lgbm
d_train = lgb.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

lg = lgb.train(params, train_set=d_train)

## Step 6: Evaluate the model

In [79]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [80]:
# Use model to predict on train and val sets
y_pred_trn_lr = lr.predict(X_train)
y_pred_val_lr = lr.predict(X_val)

y_pred_trn_rf = rf.predict(X_train)
y_pred_val_rf = rf.predict(X_val)

In [81]:
# LGBM prediction
y_pred_trn_lg = lg.predict(X_train)
y_pred_val_lg = lg.predict(X_val)
# setting threshold to .5
#size = y_pred_val_lg.size()
for i in range(0, y_pred_trn_lg.size):
    if y_pred_trn_lg[i]>=.5:       
       y_pred_trn_lg[i]=1
    else:  
       y_pred_trn_lg[i]=0
    
for i in range(0, y_pred_val_lg.size):
    if y_pred_val_lg[i]>=.5:       
       y_pred_val_lg[i]=1
    else:  
       y_pred_val_lg[i]=0
    
#print(y_pred_trn_lg)

In [82]:
print('LR train set accuracy', acc(y_train, y_pred_trn_lr))
print('LR val set accuracy', acc(y_val, y_pred_val_lr))
print('RF train set accuracy', acc(y_train, y_pred_trn_rf))
print('RF val set accuracy', acc(y_val, y_pred_val_rf))
print('LGBM train set accuracy', acc(y_train, y_pred_trn_lg))
print('LGBM val set accuracy', acc(y_val, y_pred_val_lg))

LR train set accuracy 0.800561797752809
LR val set accuracy 0.7988826815642458
RF train set accuracy 0.8764044943820225
RF val set accuracy 0.8100558659217877
LGBM train set accuracy 0.6587078651685393
LGBM val set accuracy 0.6480446927374302


## Step 7: Generate Submission

In [99]:
# Predict for test set
y_pred_test = lg.predict(X_test)
# fit predicitons to either 1 or 0
for i in range(0, y_pred_test.size):
    if y_pred_test[i]>=.5:       
       y_pred_test[i]=1
    else:  
       y_pred_test[i]=0

# cast to int
y_pred_test = y_pred_test.astype(int)

In [100]:
# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('week_3_baseline.csv', index=False)